In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Model is using 8000 positive and 8000 negative
# 4000 tweets are general positive while 4000 are airlines for positive sentiments
# 2400 are disaster  and news tweets, 3000 tweets are negative airline sentiments and 2500 general negative

Using TensorFlow backend.


In [3]:
data = pd.read_csv('clean_tweet_model9.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [4]:
test_data = pd.read_csv('news_tests.csv')
# Keeping only the neccessary columns
test_data = test_data[['text','sentiment']]

In [5]:
data['text'] = data['text'].apply(lambda x: str(x))
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)
    
max_features = 2500
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X_train = tokenizer.texts_to_sequences(data['text'].values)
X_train = pad_sequences(X_train)

22194
15800


In [6]:
test_data['text'] = test_data['text'].apply(lambda x: str(x))
test_data['text'] = test_data['text'].apply(lambda x: x.lower())
test_data['text'] = test_data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(test_data[ test_data['sentiment'] == 'Positive'].size)
print(test_data[ test_data['sentiment'] == 'Negative'].size)
    
tokenizer_test = Tokenizer(num_words=max_features, split=' ')
X_test = tokenizer.texts_to_sequences(test_data['text'].values)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])

2008
2028


In [7]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 217, 128)          320000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 217, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 575,194
Trainable params: 575,194
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
Y_train = pd.get_dummies(data['sentiment']).values
Y_test = pd.get_dummies(test_data['sentiment']).values
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(18997, 217) (18997, 2)
(2018, 217) (2018, 2)


In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 12, batch_size=batch_size, verbose = 2)

Epoch 1/12
 - 372s - loss: 0.4870 - acc: 0.7613
Epoch 2/12
 - 441s - loss: 0.3754 - acc: 0.8302
Epoch 3/12
 - 378s - loss: 0.3381 - acc: 0.8496
Epoch 4/12
 - 376s - loss: 0.3167 - acc: 0.8615
Epoch 5/12
 - 407s - loss: 0.2980 - acc: 0.8713
Epoch 6/12
 - 400s - loss: 0.2786 - acc: 0.8794
Epoch 7/12
 - 439s - loss: 0.2611 - acc: 0.8876
Epoch 8/12
 - 405s - loss: 0.2473 - acc: 0.8947
Epoch 9/12


In [9]:
X_validate = X_test
Y_validate = Y_test
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.33
acc: 0.87


In [1]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
        
    if np.argmax(result) != np.argmax(Y_validate[x]):
        print(test_data.values[x][0])
        print(test_data.values[x][1])
     
    
print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

NameError: name 'X_validate' is not defined